In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [17]:
import urllib
f = urllib.urlretrieve("https://www.dropbox.com/s/73wr8xb5s6fdj7g/apache.access.log.PROJECT?dl=1", "apache.access.log.PROJECT")

In [2]:
datos = "./apache.access.log.PROJECT"
datos_raw = sc.textFile(datos)

In [19]:
datos_raw.count()

1043177

In [20]:
datos_raw.take(5)

[u'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

In [3]:
#Vamos a tratar los datos previamente
datos_tratados = datos_raw.map(lambda x: x.replace(" -", ' 0'))\
    .map(lambda x: x.split(" "))

#Vamos a recorrer los datos imprimiendo cada columna para ver si está todo correcto
for i in range(0,10):
    datos_test = datos_tratados.map(lambda x: x[i])
    print(datos_test.take(1))

#0 es la url
#1 no es nada
#2 no es nada
#3 es la fecha - Habria que parsear con []
#4 ni idea, pero creo que nada
#5 protocolo de respuesta: GET/POST
#6 url de archivo accedido
#7 HTTP version
#8 HTTP status - Access through -2
#9 Size - Access through -1   

[u'in24.inetnebr.com']
[u'0']
[u'0']
[u'[01/Aug/1995:00:00:01']
[u'00400]']
[u'"GET']
[u'/shuttle/missions/sts-68/news/sts-68-mcc-05.txt']
[u'HTTP/1.0"']
[u'200']
[u'1839']


In [247]:
#### EJERCICIO 1 ######

#Podemos transformar los datos aplicando map al dataset para obtener solo la columna
#deseada

datos_split = datos_tratados.map(lambda x: int(x[-1]))
print(datos_split.max())
print(datos_split.min())
print(datos_split.mean())

3421948
0
17531.5557024


In [23]:
####### EJERCICIO 2 #########
### Numero de peticiones de cada codigo de respuesta

from operator import add
response_code = datos_tratados.map(lambda y: (int(y[-2]), 1))

#Muestra una lista de tuplas con valor(statusCode, count)
response_code.reduceByKey(add).sortByKey().collect()


[(200, 940847),
 (302, 16244),
 (304, 79824),
 (403, 58),
 (404, 6185),
 (500, 2),
 (501, 17)]

In [24]:
##### EJERCICIO 3 ######
# Mostrar 20 hosts que han sido visitados mas de 10 veces
hosts = datos_tratados.map(lambda y: (y[0], 1))
agrupados = hosts.reduceByKey(add)\
    .sortByKey()\
    .filter(lambda x: x[1] > 10)\
    .take(20)

#Muestra una lista de tuplas con valor(host, count)
agrupados

[(u'***.novo.dk', 16),
 (u'03-dynamic-c.wokingham.luna.net', 39),
 (u'05-dynamic-c.wokingham.luna.net', 25),
 (u'0875pr3e.pro.rpslmc.edu', 16),
 (u'1.ts1.mnet.medstroms.se', 32),
 (u'10.ts2.mnet.medstroms.se', 15),
 (u'101.irri.cgiar.org', 17),
 (u'1032015.ksc.nasa.gov', 90),
 (u'10forward.ppp.america.com', 18),
 (u'10md423.uni-duisburg.de', 39),
 (u'118.32.med.umich.edu', 15),
 (u'12.ts2.mnet.medstroms.se', 25),
 (u'120.33.med.umich.edu', 14),
 (u'128.100.180.37', 15),
 (u'128.100.183.252', 13),
 (u'128.100.183.253', 36),
 (u'128.100.191.43', 31),
 (u'128.100.197.90', 16),
 (u'128.100.8.177', 40),
 (u'128.101.13.25', 15)]

In [228]:
##### EJERCICIO 4 #####
# Mostrar los 10 endpoints mas visitados
endpoint = datos_tratados.map(lambda y: (y[6], 1))
endpoint_agr = endpoint.reduceByKey(add)\
    .sortBy(lambda x: x[1], False)\
    .take(10)

#Muestra una lista de tuplas con valor(endpoint, count)
endpoint_agr


[(u'/images/NASA-logosmall.gif', 59666),
 (u'/images/KSC-logosmall.gif', 50420),
 (u'/images/MOSAIC-logosmall.gif', 43831),
 (u'/images/USA-logosmall.gif', 43604),
 (u'/images/WORLD-logosmall.gif', 43217),
 (u'/images/ksclogo-medium.gif', 41267),
 (u'/ksc.html', 28536),
 (u'/history/apollo/images/apollo-logo1.gif', 26766),
 (u'/images/launch-logo.gif', 24742),
 (u'/', 20175)]

In [229]:
##### EJERCICIO 5 ######
# Mostrar 10 endpoints que NO tengan codigo 200
endpointNO = datos_tratados.map(lambda y: (y[6], y[-2]))
endpointNO_agr = endpointNO.sortBy(lambda x: x[1], False)\
    .filter(lambda x: x[1] != 200)\
    .take(10)

#Muestra una lista de tuplas con valor(endpoint, statusCode)
#Con la condicion del status code distinto de 200
endpointNO_agr


[(u'/ksc.html', u'501'),
 (u'/ksc.html', u'501'),
 (u'/ksc.html', u'501'),
 (u'/ksc.html', u'501'),
 (u'/ksc.html', u'501'),
 (u'/images/getstats.gif', u'501'),
 (u'/ksc.html', u'501'),
 (u'/shuttle/missions/51-i/51-i-patch-small.gif', u'501'),
 (u'/shuttle/missions/sts-45/sts-45-patch-small.gif', u'501'),
 (u'/shuttle/missions/sts-6/sts-6-patch-small.gif', u'501')]

In [230]:
##### EJERCICIO 6 ######
# Calcular el numero de hosts distintos

hosts_distinct = datos_tratados\
    .map(lambda y: y[0]).distinct().count()

#Muestra un integer con el numero de hosts distintos
hosts_distinct

54507

In [5]:
from pyspark.sql import Row, SQLContext
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

In [6]:
##### EJERCICIO 7 #####
# Calcular el numero de host unicos cada dia

row_data = datos_tratados.map(lambda p: Row(
    host = p[0], 
    empty_first = int(p[1]),
    empty_second = p[2],
    date = p[3],
    protocol = p[4],
    endpoint = p[6],
    version = p[7],
    status_Code = int(p[-2]),
    size = p[-1]
    )
)

interactions_df = sqlContext.createDataFrame(row_data)

newdf = interactions_df.withColumn('date', regexp_replace('date', '\[', ''))
newdf2 = newdf.withColumn('date', regexp_replace('date', 'Aug', '08'))

newdf3 = newdf2.select(from_unixtime(unix_timestamp('date', 'dd/MM/yyyy:hh:mm:ss')).alias('date'), 'host')

newdf4 = newdf3.na.fill(0)

#Muestra los host distintos de cada día del mes de Agosto
newdf4.groupby(dayofmonth("date")\
        .alias("Dia del mes"))\
        .agg(countDistinct('host').alias('Host distintos'))\
        .sort('Dia del mes')\
        .show()

+-----------+--------------+
|Dia del mes|Host distintos|
+-----------+--------------+
|          1|          2582|
|          3|          3222|
|          4|          4190|
|          5|          2502|
|          6|          2537|
|          7|          4106|
|          8|          4406|
|          9|          4317|
|         10|          4523|
|         11|          4346|
|         12|          2864|
|         13|          2650|
|         14|          4454|
|         15|          4214|
|         16|          4340|
|         17|          4385|
|         18|          4168|
|         19|          2550|
|         20|          2560|
|         21|          4134|
+-----------+--------------+
only showing top 20 rows



In [7]:
##### EJERCICIO 8 #####
# Calcular la media de peticiones diarias por host

data_frame = newdf4.withColumn("month", month(col("date"))).\
                                   withColumn("DayOfmonth", dayofmonth(col("date")))
data_frame.createOrReplaceTempView('data_frame')
sqlContext.sql("SELECT DATE(date) Date, COUNT(host)/COUNT(DISTINCT host)\
                AS Peticiones_diarias_host FROM data_frame GROUP BY  DATE(date) ORDER BY DATE(date) ASC").show(n = 10)

+----------+-----------------------+
|      Date|Peticiones_diarias_host|
+----------+-----------------------+
|1995-08-01|     13.166537567776917|
|1995-08-03|     12.845127250155183|
|1995-08-04|     14.213365155131266|
|1995-08-05|     12.745003996802557|
|1995-08-06|     12.777296018919984|
|1995-08-07|     13.968582562104238|
|1995-08-08|     13.650022696323196|
|1995-08-09|      14.00440120454019|
|1995-08-10|     13.540791510059695|
|1995-08-11|     14.091578462954441|
+----------+-----------------------+
only showing top 10 rows



In [232]:
##### EJERCICIO 9 #####
# Mostrar una lista de 40 endpoints distintos que tiran 404

endpoint_error = datos_tratados.map(lambda y: (y[6], int(y[-2])))
endpointerror_agr = endpoint_error.reduceByKey(add)\
    .sortBy(lambda x: x[1], False)\
    .filter(lambda x: x[1] == 404)\
    .distinct()\
    .take(40)

#Muestra una lista de tuplas con valor(endpoint, statusCode)
endpointerror_agr


[(u'/msfe/astro_home.html', 404),
 (u'/space.pub.gif', 404),
 (u'/shuttle/missions/missions,html', 404),
 (u'/facilities/rss.gif', 404),
 (u'/shuttle/technology/images/sts-comm.jpg', 404),
 (u'/history/appollo/apollo-13', 404),
 (u'/shuttle/missions/sts-67/sts-67-patch-large.gif', 404),
 (u'/hmhome.hmtl', 404),
 (u'/images/NASAlogo-small.gif', 404),
 (u'/history/apollo/aplool-13/apollo-13.html', 404),
 (u'/review.com/index.html', 404),
 (u'/software/winvn/winwn.html', 404),
 (u'/missions/missions', 404),
 (u'/hisorty/history.html', 404),
 (u'/welcome/html', 404),
 (u'/shuttle/news/sci.space.news/1295', 404),
 (u'/missions/51-l', 404),
 (u'/history/apollo/thumb/igneous.gif', 404),
 (u'/images/School05.jpg', 404),
 (u'/pub/wiinvn/win3/', 404),
 (u'/history/apollo/apollo-2/apollo-2-info.html', 404),
 (u'/shuttle/technology/sts-newsref/stsre', 404),
 (u'/shuttle/countdown/images/griffin1.gif', 404),
 (u'/shuttle/technology/sts-newsref/sts_asm.html#srb_mod_nozzle', 404),
 (u'/shuttle/missio

In [234]:
##### EJERCICIO 10 #####
# Top 25 endpoints que más 404 tiran

#Necesitamos solo el status code 404, el endpoint y un contador para agrupar dentro de la tupla
endp_filtrado = datos_tratados.filter(lambda x: int(x[-2]) == 404)

endpoint_top_error = endp_filtrado.map(lambda x: (x[6], 1))
endpointtoperror_agr = endpoint_top_error.reduceByKey(add)\
    .sortBy(lambda x: x[1], False)\
    .take(25)

endpointtoperror_agr

[(u'/pub/winvn/readme.txt', 633),
 (u'/pub/winvn/release.txt', 494),
 (u'/shuttle/missions/STS-69/mission-STS-69.html', 430),
 (u'/images/nasa-logo.gif', 319),
 (u'/elv/DELTA/uncons.htm', 178),
 (u'/shuttle/missions/sts-68/ksc-upclose.gif', 154),
 (u'/history/apollo/sa-1/sa-1-patch-small.gif', 146),
 (u'/images/crawlerway-logo.gif', 120),
 (u'/://spacelink.msfc.nasa.gov', 117),
 (u'/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif', 100),
 (u'/history/apollo/a-001/a-001-patch-small.gif', 97),
 (u'/images/Nasa-logo.gif', 85),
 (u'/shuttle/resources/orbiters/atlantis.gif', 63),
 (u'/history/apollo/images/little-joe.jpg', 62),
 (u'/images/lf-logo.gif', 59),
 (u'/shuttle/resources/orbiters/discovery.gif', 56),
 (u'/shuttle/resources/orbiters/challenger.gif', 54),
 (u'/robots.txt', 53),
 (u'/elv/new01.gif>', 43),
 (u'/history/apollo/pad-abort-test-2/pad-abort-test-2-patch-small.gif', 38),
 (u'/pub', 36),
 (u'/pub/', 36),
 (u'/history/apollo/sa-2/sa-2-patch-small.gif', 35),
 

In [8]:
##### EJERCICIO 11 #####
# Top 5 días que generaron código de respuesta 404
newdf5 = newdf2.select(from_unixtime(unix_timestamp('date', 'dd/MM/yyyy:hh:mm:ss'))\
                       .alias('date'), 'status_Code')

newdf5.filter(newdf5['status_Code'] == 404)\
        .groupBy(dayofmonth('date'))\
        .count()\
        .sort(col("count").desc())\
        .show(n = 5)

+----------------+-----+
|dayofmonth(date)|count|
+----------------+-----+
|               7|  532|
|               8|  381|
|               6|  372|
|               4|  346|
|              15|  326|
+----------------+-----+
only showing top 5 rows

